# Interactive Visualization Lab - Superstore Dataset

Completa el siguiente conjunto de ejercicios para consolidar tus conocimientos sobre la visualización interactiva utilizando Plotly.

0. Leer los datos.
1. Crea un gráfico de barras interactivo que muestre las ventas totales por Estado en 2014.
2. Crea un gráfico de líneas interactivo que muestre las ventas y los beneficios en 2014.
3. Crea un conjunto de 3 histogramas interactivos que muestren las distribuciones de cantidad por pedido para los siguientes estados: California, Nueva York y Texas.
4. Crea un gráfico de barras apiladas interactivo que muestre los ingresos por estado (barras) para cada una de las categorías de productos para California, Nueva York y Texas.
5. Crea un gráfico de líneas interactivo que muestre la venta por día para California.

In [2]:
# Importación de bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly as py
import plotly.offline as pyo
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# Biblioteca de nubes de palabras
from wordcloud import WordCloud

# Verificando el contenido del directorio "./data"
import os
print(os.listdir("./data"))

# Cualquier resultado que escribas en el directorio actual se guarda como salida.
# Ignorando advertencias para obtener una salida más limpia
import warnings
warnings.filterwarnings("ignore")


['Sample - Superstore.xlsx']


In [5]:
!pip install openpyxl

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     -------------------------------------  245.8/250.0 kB 7.6 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 7.7 MB/s eta 0:00:00


In [79]:
# Abriendo el df
df = pd.read_excel("./data/Sample - Superstore.xlsx")
df.head(5)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
# Convertir las columnas de fechas a objetos de fecha si no lo están.
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

In [65]:
#Filtramos por año usando el acceso dt --> Seccion de metodos de fecha
df['Order Date'].dt.year == 2014

0       False
1       False
2       False
3       False
4       False
        ...  
9989     True
9990    False
9991    False
9992    False
9993    False
Name: Order Date, Length: 9994, dtype: bool

In [66]:
# Filtrar los datos para obtener solo las ventas en 2014.
df_2014 = df[df['Order Date'].dt.year == 2014]

In [80]:
#Hacer el groupby por Estado y aplicar el conteo
ventas_totales = df_2014.groupby('State')[['Sales']].sum().reset_index()
ventas_totales.head(7)

,State,Sales
0,Alabama,6139.090
1,Arizona,8295.235
2,Arkansas,6302.690
3,California,91303.531
4,Colorado,6502.294
5,Connecticut,2794.020
6,Delaware,4786.030


In [78]:

# Crear trace1 
trace1 = go.Bar(x=ventas_totales['State'],  # Utilizamos el índice del DataFrame (Estados)
               y=ventas_totales['Sales'],
               name='Ventas totales por Estado',
               marker=dict(color='rgba(255, 174, 255, 0.5)',
                           line=dict(color='rgb(0,0,0)', width=1.5)),
               text=ventas_totales.index)

data = [trace1]

layout = go.Layout(barmode="group", xaxis=dict(title='Estado'), yaxis=dict(title='Ventas totales'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)




In [110]:
# Realizar el groupby por estado y sumar las ventas
ventas_totales = df_2014.groupby('Order Date')['Sales'].sum().reset_index()

# Realizar el groupby por estado y sumar los beneficios
beneficios_totales = df_2014.groupby('Order Date')['Profit'].sum().reset_index()

# Fusionar los resultados en un solo DataFrame
df_resultado = pd.merge(ventas_totales, beneficios_totales, on='Order Date', how='inner')


df_resultado.head(5)

,Order Date,Sales,Profit
0,2014-01-03,16.448,5.5512
1,2014-01-04,288.060,-65.9901
2,2014-01-05,19.536,4.8840
3,2014-01-06,4407.100,1358.0524
4,2014-01-07,87.158,-71.9621


In [112]:

# Crear un objeto de línea para las ventas totales por estado
trace_ventas = go.Scatter(x=df_resultado['Order Date'], y=df_resultado['Sales'],
                         mode='lines+markers', name='Ventas Totales',
                         line=dict(color='blue', width=2))

# Crear un objeto de línea para los beneficios totales por estado
trace_beneficios = go.Scatter(x=df_resultado['Order Date'], y=df_resultado['Profit'],
                              mode='lines+markers', name='Beneficios Totales',
                              line=dict(color='green', width=2))

# Crear layout
layout = go.Layout(title='Ventas y Beneficios Totales 2014 por Estado',
                   xaxis=dict(title='Estado'),
                   yaxis=dict(title='Monto'),
                   showlegend=True)

# Crear figura y añadir trazas
fig = go.Figure(data=[trace_ventas, trace_beneficios], layout=layout)

# Mostrar el gráfico
iplot(fig)


In [19]:
print(df_resultado.columns)



Index(['Sales', 'Profit'], dtype='object')


In [93]:
df_California = df[df.State== 'California']
Q_Pedidos_California = df_California.groupby('Order ID')['Quantity'].sum()
Q_Pedidos_California

Order ID
CA-2014-100090     9
CA-2014-100860     5
CA-2014-100867     6
CA-2014-101462     4
CA-2014-101931    17
                  ..
US-2017-160143     3
US-2017-163300    17
US-2017-163657     7
US-2017-163790    31
US-2017-167318     8
Name: Quantity, Length: 1021, dtype: int64

In [41]:
df_Nueva_York = df[df['State']== 'New York']
Q_Pedidos_Nueva_York = df_Nueva_York.groupby('Order ID')['Quantity'].sum()
Q_Pedidos_Nueva_York

Order ID
CA-2014-100006     3
CA-2014-100328     1
CA-2014-100391     2
CA-2014-101364    13
CA-2014-101476     1
                  ..
US-2017-148866     5
US-2017-155866    10
US-2017-157896     3
US-2017-166037     9
US-2017-168613     2
Name: Quantity, Length: 562, dtype: int64

In [42]:
df_Texas = df[df['State']== 'Texas']
Q_Pedidos_Texas = df_Texas.groupby('Order ID')['Quantity'].sum()
Q_Pedidos_Texas

Order ID
CA-2014-100678    11
CA-2014-101602     8
CA-2014-103086     2
CA-2014-103492    19
CA-2014-103744     6
                  ..
US-2017-152366     4
US-2017-155299     2
US-2017-156356     6
US-2017-158218     9
US-2017-160836    10
Name: Quantity, Length: 487, dtype: int64

In [75]:


trace1 = go.Histogram(x = Q_Pedidos_Texas,
                     opacity = 0.9,
                     name = 'Texas',
                     marker = dict(color = 'rgba(171,50,96,0.6)'))

trace2 = go.Histogram(x = Q_Pedidos_California,
                     opacity = 0.9,
                     name = 'California',
                     marker = dict(color = 'rgba(12,50,196,0.6)'))

trace3 = go.Histogram(x = Q_Pedidos_Nueva_York,
                     opacity = 0.9,
                     name = 'Nueva York',
                     marker = dict(color = 'rgba(80,90,196,0.6)'))

data = [trace1, trace2, trace3]
layout = go.Layout(barmode='overlay',
                   title=' Numero de pedidos por Estado',
                   xaxis=dict(title='Pedidos'),
                   yaxis=dict( title='Numero'),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [59]:
ventas_categoria_CA= df_California.groupby('Category')[['Sales']].sum().reset_index()
ventas_categoria_NY = df_Nueva_York.groupby('Category')[['Sales']].sum().reset_index()
ventas_categoria_TX = df_Texas.groupby('Category')[['Sales']].sum().reset_index()

print(ventas_categoria_TX)
print(ventas_categoria_NY)
print(ventas_categoria_CA)

          Category       Sales
0        Furniture  60593.2918
1  Office Supplies  44490.5300
2       Technology  65104.2240
          Category       Sales
0        Furniture   93372.729
1  Office Supplies   90020.042
2       Technology  127483.500
          Category        Sales
0        Furniture  156064.6015
1  Office Supplies  142351.9480
2       Technology  159271.0820


In [74]:

trace1 = {
    'x': ventas_categoria_TX['Category'],
    'y': ventas_categoria_TX['Sales'],
    'name': 'Ventas de Texas',
    'type': 'bar'
}

trace2 = {
    'x': ventas_categoria_NY['Category'],
    'y': ventas_categoria_NY['Sales'],
    'name': 'Ventas de Nueva York',
    'type': 'bar'
}

trace3 = {
    'x': ventas_categoria_CA['Category'],
    'y': ventas_categoria_CA['Sales'],
    'name': 'Ventas de California',
    'type': 'bar'
}
data = [trace1, trace2, trace3]

layout = {
    'xaxis': {'title': 'Top 3 Estados de USA'},
    'barmode': 'group',
    'title': 'Ventas por categoria en el top 3 estados'
}

fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [58]:
ventas_dia_CA = df_California.groupby('Order Date')[['Sales']].sum().reset_index()
ventas_dia_CA

,Order Date,Sales
0,2014-01-06,19.440
1,2014-01-13,1679.749
2,2014-01-20,38.660
3,2014-01-27,426.670
4,2014-01-31,290.666
...,...,...
654,2017-12-24,2970.432
655,2017-12-25,424.172
656,2017-12-28,1066.544
657,2017-12-29,797.064


In [73]:

# Crear trace1 
trace1 = go.Bar(x=ventas_dia_CA['Order Date'],  # Utilizamos el índice del DataFrame (Estados)
               y=ventas_dia_CA['Sales'],
               name='Ventas diarias en California',
               marker=dict(color='rgba(255, 174, 255, 0.5)',
                           line=dict(color='rgb(0,0,0)', width=1.5)),
               text=ventas_dia_CA)

data = [trace1]

layout = go.Layout(barmode="group", xaxis=dict(title='Ventas diarias'), yaxis=dict(title='Ingresos'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)